In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
import glob
import zipfile

/var/folders/3l/5v7rvfhn4sn35nr7f47zf2kr0000gn/T/ipykernel_67439/3717543875.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#unzip files
filenames = glob.glob('*zip')
print(filenames)
for file in filenames:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("apartments_pl")

['apartments_pl_2024_02.csv.zip', 'apartments_pl_2023_09.csv.zip', 'apartments_pl_2023_10.csv.zip', 'apartments_pl_2023_11.csv.zip', 'apartments_pl_2023_08.csv.zip', 'apartments_pl_2024_01.csv.zip', 'apartments_pl_2023_12.csv.zip']


In [3]:
#load files 
filenames = glob.glob('apartments_pl/*csv')
df = pd.DataFrame()
for file in filenames:
    df_temp = pd.read_csv(file)
    df = pd.concat([df, df_temp])
df.shape[0]

117259

In [4]:
#Somehow merge the duplicated id's
list = df.duplicated(['id'])
type(list)
list.tail()

16356    True
16357    True
16358    True
16359    True
16360    True
dtype: bool

In [5]:
#drop observations
df = df.dropna()

#convert to numerical data
df['condition'] = df['condition'].map({'low': 0, 'premium': 1})
df['type'] = df['type'].map({'apartmentBuilding': 1, 'blockOfFlats': 2, 'tenement': 3})
for col in ['hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom']:
    df[col] = df[col].map({'no': 0, 'yes': 1})

for col in ['hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom', 'condition', 'type']:
    df[col] = df[col].astype(np.int8)

#drop features
toDrop = ['id', 'ownership'] #buildMaterial, city
df = df.drop(toDrop, axis=1)


In [ ]:
#drop low variance featurse
sel = VarianceThreshold(threshold=.05)
sel.fit(df/df.mean())
mask = sel.get_support() 
#Still need longtude, latitude, and buildYear
df = df.loc[:, mask]

In [ ]:
df.info()
df.describe()

In [6]:
df.to_csv('full_dataset.csv', index=False)